In [ ]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [ ]:
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
from my_code import pytorch_model as m

In [ ]:
# name of the notebook
name_notebook = "sweep_12c_h"

In [ ]:
sweep = c.Sweep(
    name_notebook, 
    initial_path=initial_path,
    description="hot encoding classical network of 3 layers",
    optimizer=[m.optim.Adam, m.optim.SGD],
    lr=[1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5],
)

In [ ]:
sweep.save()

In [ ]:
for sweep_point in sweep:

    # print the point of the sweep
    sweep.print_sweep_status(sweep_point['idx'])

    # -- MODEL -- #
    class Model(m.Model):
        def __init__(self):
            super(Model, self).__init__()
            self.fc1 = m.nn.Linear(228, 228//2)
            self.fc2 = m.nn.Linear(228//2, 12)
            self.fc3 = m.nn.Linear(12, 1)

        def forward(self, x):
            x = m.torch.sigmoid(self.fc1(x))
            x = m.torch.sigmoid(self.fc2(x))
            x = self.fc3(x)
            return x

    model = Model()

    # set the name and path of the model
    model.set_name_and_path(name_notebook, initial_path)

    #set sweep point
    model.set_sweep_point(
        sweep_uuid = sweep.uuid,
        sweep_point = sweep_point
    )

    # -- TRAIN -- #
    model.optimize_params(
        #save info
        save_model_info = True,
        description = 'sweep test with 12 amino acids and clasical QML',    
        #data
        data=c.Data.load(initial_path=initial_path, file_name='PET_SCORES_12'),
        # train options
        optimizer = sweep_point['optimizer'],
        optimizer_options = {'lr': sweep_point['lr']},
        num_epochs = 2,                
        batch_size = 32,
        # early stopping options
        stop_training_options = {
            'lookback_epochs': 5, 
            'threshold_slope': 0.001,
            'threshold_std_dev': 0.2
        },
        # metadata
        metadata = {}
    )

    # -- VALIDATION and PLOTS -- #
    model.plot_losses(fig_size=(6,6))
    model.validate(
        pct=1,
        add_to_results=True,
        plot=True,
        print_items=False
    )

    # -- SAVE RESULTS -- #
    model.save_results()

    # -- ADD DATA GENERATED TO SWEEP -- #
    sweep.add_data(
        idx=sweep_point['idx'],
        model_uuid=model.uuid,
        best_loss_train=model.results.loss_epoch.best,
        best_loss_test=model.results.loss_validation_epoch.best,
        loss_validation=model.mean_loss_validation,
        time_train=model.results.time_epoch.last,
    )
    sweep.save(csv=False) #TODO: save csv=False automatically when sweep is already saved ion csv

    